In [28]:
import matplotlib.pyplot as plt
import pandas as pd
from recallibrated_data_moving. util import *
import numpy as np
from scipy.stats.stats import pearsonr
import os
import pathlib


C:\Users\satya\AppData\Local\Temp\ipykernel_13148\2051684909.py:5: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [29]:
subfolders = [ f.name for f in os.scandir(os.getcwd()) if f.is_dir() ]
recallibrated_dir = 'recallibrated_data_moving'
exclusions = ['data_validate', 'recallibrated_data_moving', 'recallibrated_data_static', 'pilot_testing']
precision_saccade_df = pd.DataFrame(columns=['Name','ISI', 'precision_saccade', 'precision_fixate']) 
precision_smooth_df = pd.DataFrame(columns=['Name', 'precision_smooth', 'precision_fixate']) 

error_df_path = recallibrated_dir + '/' +'error_df.csv'
precision_saccade_df_path = 'precision_saccade_df.csv'
precision_smooth_df_path = 'precision_smooth_df.csv'
error_df = pd.read_csv(error_df_path)

confidence_threshold = 0.6

if os.path.exists(precision_saccade_df_path):
        precision_saccade_df = pd.read_csv(precision_saccade_df_path)
if os.path.exists(precision_smooth_df_path):
    precision_smooth_df = pd.read_csv(precision_smooth_df_path)

for folder in subfolders:
    desktop = pathlib.Path(folder)
    files = [f.name for f in desktop.iterdir() if f.is_file()]
    data_files = []
    callibration_file = 'not_found'
    
    for f in files:
        if "callibrate" in f:
            callibration_file = folder+"/"+f
        elif "csv" in f:
            data_files.append(folder+"/"+f)

    if callibration_file == 'not_found' or folder in exclusions:
        continue
    
    all_files_callibrated = True
    for f in files: 
        all_files_callibrated = all_files_callibrated and os.path.exists(recallibrated_dir + '/' + f[:-4] + '_recal.csv') 

    if not (all_files_callibrated and folder in error_df['Name'].values) or \
        error_df.loc[error_df['Name'] == folder].callibrate_vec.astype(float).item()>2.5:            
        continue

    # Using the callibration task for saccadic/fixations
    if folder not in precision_saccade_df['Name'].values:
        data_frame = pd.read_csv(callibration_file, index_col=False)  
        df = clean_df(data_frame)
        rms_error_saccadic = 0
        rms_error_fixate = 0
        count_saccadic = 0
        count_fixate = 0
        isi = df.iloc[1]['seconds'] - df.iloc[0]['seconds']

        for index, row in df[1:].iterrows():
            isi += row['seconds'] - df.iloc[index-1]['seconds']
            if row['path']==99 and row['gaze_confidence'] < confidence_threshold:#Saccade
                p = [df.iloc[index-1]['gaze_vis_x'], df.iloc[index-1]['gaze_vis_y']]
                q = [row['gaze_vis_x'],row['gaze_vis_y']]
                rms_error_saccadic += math.dist(p, q)**2
                count_saccadic+=1
            if row['path']!=99 and row['gaze_confidence'] > confidence_threshold:#Fixate
                p = [df.iloc[index-1]['gaze_vis_x'], df.iloc[index-1]['gaze_vis_y']]
                q = [row['gaze_vis_x'],row['gaze_vis_y']]
                rms_error_fixate += math.dist(p, q)**2
                count_fixate+=1

        if not (count_fixate==0 or count_saccadic==0):
            rms_error_saccadic = math.sqrt(rms_error_saccadic/count_saccadic)
            rms_error_fixate = math.sqrt(rms_error_fixate/count_fixate)
            isi /= len(df.index)-1
            precision_saccade_df.loc[len(precision_saccade_df.index)] = [folder, isi, rms_error_saccadic, rms_error_fixate] 
            precision_saccade_df.to_csv(precision_saccade_df_path, index= False)

    #Using w1 for smooth pursuit task
    if folder not in precision_smooth_df['Name'].values:
        w1_file = [word for word in data_files if "w1" in word][0]
        data_frame = pd.read_csv(w1_file, index_col=False)  
        df = clean_df(data_frame, False)
        rms_error_smooth = 0
        count_smooth = 0
        rms_error_fixate = 0
        count_fixate = 0
        for index, row in df[1:].iterrows():
            if row['path']!=99 and row['path']>0 and row['gaze_confidence'] > confidence_threshold:#Smooth Pursuit
                p = [df.iloc[index-1]['gaze_vis_x'], df.iloc[index-1]['gaze_vis_y']]
                q = [row['gaze_vis_x'],row['gaze_vis_y']]
                rms_error_smooth += math.dist(p, q)**2
                count_smooth+=1
            if row['path']<0 and row['gaze_confidence'] > confidence_threshold:#Fixate
                p = [df.iloc[index-1]['gaze_vis_x'], df.iloc[index-1]['gaze_vis_y']]
                q = [row['gaze_vis_x'],row['gaze_vis_y']]
                rms_error_fixate += math.dist(p, q)**2
                count_fixate+=1

        if  not (count_fixate==0 or count_smooth==0):
            rms_error_smooth = math.sqrt(rms_error_smooth/count_smooth)
            rms_error_fixate = math.sqrt(rms_error_fixate/count_fixate)
            precision_smooth_df.loc[len(precision_smooth_df.index)] = [folder, rms_error_smooth, rms_error_fixate] 
            precision_smooth_df.to_csv(precision_smooth_df_path, index= False)
    
    

In [30]:
import scipy
from scipy.stats.mstats import winsorize


precision_saccade_df = pd.read_csv(precision_saccade_df_path, index_col=False)
precision_smooth_df = pd.read_csv(precision_smooth_df_path, index_col=False)

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

precision_saccade_df['ISI'] = precision_saccade_df['ISI'].apply(lambda p: p*1000)
# winsorize(precision_smooth_df['precision_smooth'], limits=(0.2,0.2), inplace=True)
# winsorize(precision_smooth_df['precision_fixate'], limits=(0.2,0.2), inplace=True)
# winsorize(precision_saccade_df['precision_fixate'], limits=(0.2,0.2), inplace=True)
# winsorize(precision_saccade_df['precision_saccade'], limits=(0.2,0.2), inplace=True)

isi = mean_confidence_interval(precision_saccade_df['ISI'])
precision_smooth = mean_confidence_interval(precision_smooth_df['precision_smooth'])
precision_fixate_smooth = mean_confidence_interval(precision_smooth_df['precision_fixate'])
precision_saccade = mean_confidence_interval(precision_saccade_df['precision_saccade'])
precision_fixate_saccade = mean_confidence_interval(precision_saccade_df['precision_fixate'])
overall_list = pd.concat([precision_smooth_df['precision_smooth'],precision_smooth_df['precision_fixate'],precision_saccade_df['precision_saccade']], axis=0)
overall = mean_confidence_interval(overall_list)

print('(' + ', '.join(('%.4f' % f) for f in isi) + ')')
print('(' + ', '.join(('%.4f' % f) for f in precision_smooth) + ')')
print('(' + ', '.join(('%.4f' % f) for f in precision_fixate_smooth) + ')')
print('(' + ', '.join(('%.4f' % f) for f in precision_saccade) + ')')
print('(' + ', '.join(('%.4f' % f) for f in precision_fixate_saccade) + ')')
print('(' + ', '.join(('%.4f' % f) for f in overall) + ')')

(16.6793, 16.5314, 16.8273)
(0.4774, 0.3716, 0.5833)
(0.4841, 0.3346, 0.6336)
(3.6599, 2.9703, 4.3496)
(0.6029, 0.4848, 0.7209)
(1.5405, 1.1369, 1.9441)
